**Regressão para prever precos de carros**

**Importações iniciais**

In [0]:
import pandas as pd

**Leitura da base de dados**

In [0]:
df = pd.read_csv('autos.csv',encoding = 'ISO-8859-1')

In [0]:
df

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


**Pré-processamento dos dados**

***Removendo colunas que não influenciam na target***

In [0]:
df = df.drop('dateCrawled', axis = 1)
df = df.drop('dateCreated', axis = 1)
df = df.drop('nrOfPictures', axis = 1)
df = df.drop('postalCode', axis = 1)
df = df.drop('lastSeen', axis = 1)

In [0]:
df.shape

(371528, 15)

***Análise na coluna name***

In [0]:
df['name'].value_counts()

Ford_Fiesta                                           657
BMW_318i                                              627
Opel_Corsa                                            622
Volkswagen_Golf_1.4                                   603
BMW_316i                                              523
                                                     ... 
E46_Coupe_330ci_Facelift_mit_Neulack_und_Gasanlage      1
Kaufe_alle_Fahrzeuge_aller_Art                          1
Ford_Focus_2004_Motor_getriebe_tip_top                  1
Opel_g_astra                                            1
Wegen_Vergroesserung_muss_er_weg                        1
Name: name, Length: 233531, dtype: int64

A coluna name apresenta muita variedade nos nomes dos carros, o que pode afetar o desempenho do algoritmo. Nesse caso, a melhor opção é apagá-la.

In [0]:
df = df.drop('name', axis = 1)

***Análise da coluna seller***

In [0]:
df['seller'].value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [0]:
df = df.drop('seller', axis = 1)

***Análise da coluna offerType***

In [0]:
df['offerType'].value_counts()

Angebot    371516
Gesuch         12
Name: offerType, dtype: int64

In [0]:
df = df.drop('offerType', axis = 1)

***Correção de valores inconsistentes***

In [0]:
i1 = df.loc[df.price <= 10]

In [0]:
i1.describe()

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration
count,12118.000000,12118.000000,12118.000000,12118.000000,12118.000000
mean,0.196072,2008.717033,80.937944,122969.962040,3.650355
std,0.964849,263.528853,229.868005,49675.277206,4.029596
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,0.000000,1996.000000,0.000000,125000.000000,0.000000
50%,0.000000,2000.000000,75.000000,150000.000000,2.000000
75%,0.000000,2005.000000,120.000000,150000.000000,7.000000
max,10.000000,9999.000000,15016.000000,150000.000000,12.000000


In [0]:
df = df[df.price > 10]

In [0]:
df.shape

(359410, 12)

In [0]:
i2 = df.loc[df.price > 350000]

In [0]:
i2

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000,12,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000,4,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000,2,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000,0,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000,3,benzin,porsche,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
364171,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein
365461,599000,control,coupe,1980,manuell,377,andere,5000,3,benzin,bmw,nein
366653,99999999,control,cabrio,1996,manuell,192,3er,150000,0,NaN,bmw,NaN
366861,3895000,test,coupe,2006,NaN,799,NaN,5000,4,benzin,sonstige_autos,nein


In [0]:
df = df[df.price < 350000]

In [0]:
df.shape

(359291, 12)

***Tratamento de valores faltantes***

In [0]:
df.loc[pd.isnull(df['vehicleType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [0]:
df['vehicleType'].value_counts() #limousine

limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64

In [0]:
df.loc[pd.isnull(df['gearbox'])] 

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371443,3300,control,kombi,2006,NaN,0,touran,150000,7,diesel,volkswagen,NaN
371460,3500,control,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN
371486,350,control,kleinwagen,1996,NaN,65,punto,150000,0,NaN,fiat,NaN
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN


In [0]:
df['gearbox'].value_counts() #manuell

manuell      266547
automatik     75508
Name: gearbox, dtype: int64

In [0]:
df.loc[pd.isnull(df['model'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371399,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,fiat,ja
371476,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,sonstige_autos,ja
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein


In [0]:
df['model'].value_counts() #golf

golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: model, Length: 251, dtype: int64

In [0]:
df.loc[pd.isnull(df['fuelType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371496,3850,test,cabrio,2006,manuell,108,2_reihe,125000,2,NaN,peugeot,nein
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [0]:
df['fuelType'].value_counts() #benzin

benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64

In [0]:
df.loc[pd.isnull(df['notRepairedDamage'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371507,5999,test,kombi,2005,manuell,140,a4,150000,4,diesel,audi,NaN
371514,999,control,cabrio,2000,manuell,95,megane,150000,4,benzin,renault,NaN
371515,1690,test,kombi,2004,manuell,55,fabia,150000,4,benzin,skoda,NaN
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN


In [0]:
df['notRepairedDamage'].value_counts() #nein

nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64

Dicionário para informar quais campos terão os valores faltantes substituidos pela moda

In [0]:
valores = {'vehicleType' : 'limousine',
           'gearbox' : 'manuell',
           'model' : 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}

In [0]:
valores

{'fuelType': 'benzin',
 'gearbox': 'manuell',
 'model': 'golf',
 'notRepairedDamage': 'nein',
 'vehicleType': 'limousine'}

In [0]:
df = df.fillna(value = valores)

In [0]:
df.isnull()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,False,False,False,False,False,False,False,False,False,False,False,False
371524,False,False,False,False,False,False,False,False,False,False,False,False
371525,False,False,False,False,False,False,False,False,False,False,False,False
371526,False,False,False,False,False,False,False,False,False,False,False,False


***Label Encoder***

A função Label Encoder é utilizada para transformar dados categóricos em dados numéricos, para que o algoritmo consiga realizar todos os seus cálculos matemáticos

In [0]:
features = df.iloc[:,1:13].values

In [0]:
preco_real = df.iloc[:, 0].values

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder_features = LabelEncoder()

features[:,0] = label_encoder_features.fit_transform(features[:,0])
features[:,1] = label_encoder_features.fit_transform(features[:,1])
features[:,3] = label_encoder_features.fit_transform(features[:,3])
features[:,5] = label_encoder_features.fit_transform(features[:,5])
features[:,8] = label_encoder_features.fit_transform(features[:,8])
features[:,9] = label_encoder_features.fit_transform(features[:,9])
features[:,10] = label_encoder_features.fit_transform(features[:,10])

In [0]:
features

array([[1, 6, 1993, ..., 1, 38, 1],
       [1, 3, 2011, ..., 3, 1, 0],
       [1, 7, 2004, ..., 3, 14, 1],
       ...,
       [1, 1, 1996, ..., 3, 38, 1],
       [1, 5, 2002, ..., 3, 38, 1],
       [0, 6, 2013, ..., 1, 2, 1]], dtype=object)

***ColumnTransformer*** *italicized text*

A função One hot encoder é utilizada para criação de variavéis do tipo Dummy, para categorias.

In [0]:
# 0 0 0 0 categoria 0
# 2 0 1 0 categoria 2
# 3 0 0 1 categoria 3

In [0]:
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])],remainder='passthrough') 
features = onehotencoder.fit_transform(features).toarray() 


Validação Cruzada

In [0]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor

In [0]:
from tensorflow.keras.models import Sequential
import tensorflow as tf 

In [0]:
def criarRede():
  regressor = Sequential([
        tf.keras.layers.Dense(units=158, activation = 'relu', input_dim=316),
        tf.keras.layers.Dense(units=158, activation = 'relu'),
        tf.keras.layers.Dense(units=1, activation = 'linear')])

  regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                  metrics = ['mean_absolute_error'])
  
  return regressor

In [0]:
regressor = KerasRegressor(build_fn=criarRede,
                           epochs = 20,
                           batch_size = 100)

In [0]:
resultados = cross_val_score(estimator=regressor, X = features, y = preco_real,
                             cv = 10, scoring = 'neg_mean_absolute_error')

Epoch 1/20
3234/3234 [==============================] - 7s 2ms/step - loss: 3632.0039 - mean_absolute_error: 3632.0042
Epoch 2/20
3234/3234 [==============================] - 7s 2ms/step - loss: 2992.1492 - mean_absolute_error: 2992.1492
Epoch 3/20
3234/3234 [==============================] - 7s 2ms/step - loss: 2832.4019 - mean_absolute_error: 2832.4019
Epoch 4/20
3234/3234 [==============================] - 7s 2ms/step - loss: 2783.1755 - mean_absolute_error: 2783.1755
Epoch 5/20
3234/3234 [==============================] - 8s 2ms/step - loss: 2726.2473 - mean_absolute_error: 2726.2473
Epoch 6/20
3234/3234 [==============================] - 7s 2ms/step - loss: 2682.7930 - mean_absolute_error: 2682.7927
Epoch 7/20
3234/3234 [==============================] - 7s 2ms/step - loss: 2632.4980 - mean_absolute_error: 2632.4980
Epoch 8/20
3234/3234 [==============================] - 7s 2ms/step - loss: 2597.5486 - mean_absolute_error: 2597.5486
Epoch 9/20
3234/3234 [==========================

In [0]:
resultados

array([-2353.36406178, -2546.53152365, -2542.82706569, -2470.36422754,
       -2357.18808111, -2358.34957178, -2509.24897373, -2506.65725399,
       -2484.13891935, -2369.3511103 ])

In [0]:
resultados.mean()

-2449.8020788918416

In [0]:
resultados.std()

76.87136111441043